## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 11
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_11


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 23 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,10,2021,29.3,7.9,284.3,103.2,11.4,133.8,21.4,...,12.0,103.7,17.2,13.1,12.1,1.3,1.512712,0.113402,2.2,2.750
1,Tampa Bay Buccaneers,10,2021,30.9,7.5,314.8,104.2,11.1,91.2,22.2,...,9.6,78.4,14.1,12.7,12.6,1.4,1.907767,0.131707,2.9,3.625


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[10, 11]

In [7]:
# X[X['games_played'] == 8]

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
8,Arizona Cardinals,2021,week_11,0.252719,0.220395,0.299511,0.266243,0.259717
1,Tampa Bay Buccaneers,2021,week_11,0.243838,0.195258,0.271862,0.286044,0.249250
5,Los Angeles Rams,2021,week_11,0.240087,0.155579,0.264129,0.274907,0.233676


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
8,Arizona Cardinals,2021,week_11,0.252719,0.220395,0.299511,0.266243,0.259717
1,Tampa Bay Buccaneers,2021,week_11,0.243838,0.195258,0.271862,0.286044,0.249250
5,Los Angeles Rams,2021,week_11,0.240087,0.155579,0.264129,0.274907,0.233676
4,Buffalo Bills,2021,week_11,0.234615,0.130257,0.170307,0.249447,0.196157
0,Dallas Cowboys,2021,week_11,0.212810,0.147075,0.143457,0.220811,0.181038
16,New England Patriots,2021,week_11,0.154315,0.159685,0.124847,0.216349,0.163799
9,Las Vegas Raiders,2021,week_11,0.158514,0.103331,0.122408,0.221170,0.151356
17,Green Bay Packers,2021,week_11,0.135401,0.080189,0.098851,0.219007,0.133362
12,San Francisco 49ers,2021,week_11,0.119143,0.064031,0.144782,0.202687,0.132661
6,Minnesota Vikings,2021,week_11,0.108451,0.055908,0.102798,0.190566,0.114431
